### Sedaro API "Rolling Simulations" Example

This notebook shows a powerful use case of the Sedaro API where simulations are started and terminated on a rolling basis to inject incremental state corrections (i.e. from telemetry) over time.

This is particularly useful to operations use cases where telemetry can tune the simulation to match the real system(s) - enabling the simulation to predict future performance and fill telemetry gaps.

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

#### Settings and User-Specific Configuration

In [ ]:
import json
from datetime import datetime
import time
from random import random

import requests
from sedaro import SedaroApiClient, SedaroSimulationResult
from IPython import display
from tabulate import tabulate


with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('./config.json', 'r') as file:
    config = json.load(file)

# TODO: Obtain these IDs from the branch list within each repository and add to config.json
AGENT_TEMPLATE_BRANCH_ID = config['AGENT_TEMPLATE_BRANCH_ID']    # ID of the Wildfire vehicle template branch
SCENARIO_BRANCH_ID = config['SCENARIO_BRANCH_ID']                # ID of the Wildfire scenario template branch
HOST = config['HOST']                                            # Sedaro instance URL
WILDFIRE_AGENT_ID = 'NT06aqHUT5djI1_JPAsck'
WILDFIRE_FLIGHT_COMPUTER_ID = 'NT4VO8jS_HPyoLi2pu1tF'
FUTURE_OFFSET = 2/24 # 2 hours in units of days

#### Utility Functions

In [ ]:
def now_mjd(): return datetime.utcnow().timestamp() / 86400 + 40587
def mjd_to_datetime(mjd): return datetime.utcfromtimestamp(round((mjd - 40587) * 86400))
def url(scenario): return f'https://satellite.sedaro.com/#/scenario/{scenario.id}/edit/time'
def pretty_print(obj): print(json.dumps(obj, sort_keys=True, indent=2))

def summarize_tlm(now, soc, v_batt, i_batt, t_fc, t_bpa, t_bpb, annotation):
    print(f'Telemetry @ {mjd_to_datetime(now)} UTC ({annotation}):')
    print(tabulate(
        [
            ('Battery State-of-Charge', soc*100, '%'),
            ('Battery Voltage', v_batt, 'V'),
            ('Battery Current', i_batt, 'A'),
            ('Flight Computer Temperature',  t_fc, 'deg. C'),
            ('Battery Pack A Temperature',  t_bpa, 'deg. C'),
            ('Battery Pack B Temperature',  t_bpb, 'deg. C'),
        ],
        ['Label', 'Value', 'Units'],
        tablefmt="github"
    ))

#### Introduction to Modeling in Sedaro

In Sedaro, Branches most fundamentally define Templates - either an Agent Template or a Scenario Template. These Templates then define reusable, instantiatable definitions for an Agent or Scenario as follows:

- **Agent:** An actor within a simulation.

- **Scenario:** Defines the simulation, including the involved Agents and which Templates define them.

Therefore:

- **Agent Template:** Captures the model for a system.

- **Scenario Template:** Captures the instantiation of differentiated Agents, the resolution of their abstract "Targets", and the configuration of the simulation clock.

Templates are defined in SedaroML. **SedaroML** is a modeling language for defining system properties and structure as normalized, interrelated, and hierarchical blocks of attributes. SedaroML is JSON-based and is designed to be easily human and machine readable/writeable.  This includes model interpretation, traversal, etc. The SedaroML model with all meta attributes is called a `Metamodel`.  A template `Metamodel` is located under the `data` key of a Repository's Branch.

There are some specific concepts in SedaroML that are important to understand:
  - **Attributes:** Individual properties of a Block (e.g., `mass`, `voltage`, etc.) captured as a key-value pair.
  
  - **Root:** The `root` refers to the highest level key/value pairs of the JSON-object of SedaroML. The keys are attributes, just like on `block`s, but `root` is not a `block` itself. All `Metamodel`s contain `blocks` and `index` attributes in `root` which store and help with querying for all `block`s in the `Metamodel`.

  - **Blocks:** A `block` is a set of Attributes, including a `type` which defines the type of the `block` (e.g., `ReactionWheel`, `Battery`, etc.). All `block`s are located under the `blocks` attribute of the model.

  - **Hierarchy:** The `block`s in a SedaroML model have hierarchy such that a `block` `type` can extend one or more `block` `type`s (e.g., a `ReactionWheel` is a specialized `Actuator` and an `Actuator` is a specialized `Component`). The hierarchy of a model can be interpreted using the `index` and `_supers` meta attributes of a `Metamodel`.  `index` provides a lookup to traverse from a `block` `type` to its sub-`block` `type`s, ultimately down to the individual `block` instances in the model. `_supers` provides a lookup to traverse from a `block` `type` to its super-`block` `type`s.

  - **Relationships:** The model Root and its Blocks can be related to one another using Relationship Attributes.  The `_relationships` meta attribute provides a lookup from `block` `type` to its relationships Attributes.

  - **Quantity Kinds:** In SedaroML, an Attribute that has a value and a unit is called a "Quantity". Quantities may be composed of other Quantities, called Compound Quantities. A category of Quantities that share the same unit system is called a "Quantity Kind".  If a model Attribute is an explicit Quantity, it will be included in the `_quantityKinds` meta attribute lookup.  Attributes that are Explicit Quantity Kinds may be defined in any of the supported units for the Quantity Kind. For example, all angle Attributes in SedaroML may be defined in either degrees or radians. If the unit isn't provided, the default unit for the given Quantity Kind is assumed.

SedaroML is used to define both AgentTemplates and Scenario models, with Scenario models referencing AgentTemplate Branches (and therefore their models) via the `templateRef` attribute of an `Agent` Block.


In [ ]:
# Initialize Sedaro API client
sedaro_client = SedaroApiClient(api_key=API_KEY, host=HOST)

# Access Scenario SedaroML MetaModel
scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)
scenario_metamodel = scenario.data
pretty_print(scenario_metamodel)

# Access AgentTemplate SedaroML MetaModel
wildfire_agent_template = sedaro_client.get_branch(AGENT_TEMPLATE_BRANCH_ID)
agent_metamodel = wildfire_agent_template.data

##### Time-Variable Models

Using a Sedaro Results API, a SedaroML model can be fetched for a given Agent at a given timestamp.  To generate this model, each stream for that Agent is searched for the value at the given timestamp.  If the exact timestamp does not exist in the series, the most immediate previous value is used (also known as the value "last observed carried forward").

##### Differentiating Models

When using `differentiateState` to differentiate an Agent in a Scenario, pass a complete or subset SedaroML model.  This model will be merged with the Agent Template model, with the differentiating model taking precedence.  This allows for an Agent to be defined with a Template, and then differentiated with specific attributes. Or in the case of Rolling Simulations, a new simulation can be created using the complete state of an Agent from a prior simulation.

#### Approach

In order to achieve the rolling simulations use case, these modeling principles will be leveraged. 

**Note:** It is often confusing to discuss "time" with regards to simulations.  Here, "wall time" refers to the actual, real-world time and "simulation time" refers to time in the simulation.  For example, we could run a simulation for 24-hours of simulation time, and it take 20-minutes of wall time.

**The rolling simulations approach is as follows:**
1. Given an existing Scenario and associated Agent Templates, start a simulation where the simulation start time is the current wall time and set it to run for a fixed amount of simulation time into the future (e.g., 2-hours).
1. Fetch the results of the simulation periodically, as the simulation progresses, and extract specific state frames at the current wall time as simulated telemetry.
1. As real telemetry is collected during a downlink, save the full state of the Agent at the timestamp of the real telemetry (by accessing the time-variable MetaModel from the Results API) and then merge the telemetry into the MetaModel to override any telemetry-defined state.
1. Replace the current simulation with a new simulation that starts from the telemetry timestamp and that uses the telemetry-informed MetaModel as the initial state of Agent. This is done via `Agent.differentiateState`.

Cool - now lets do it!

##### 1. Start a Simulation

Start a simulation from "now" for 2 hours into the future. This simulation is initialized using the latest TLE for the ISS from CelesTrak.  Before the simulation begins, Sedaro will propagate this TLE forward to the start time of the simulation.

In [ ]:
# Get latest TLEs from CelesTrak
response = requests.get('https://celestrak.org/NORAD/elements/gp.php?GROUP=stations&FORMAT=tle')
iss_tle = '\n'.join(response.text.split('ISS (ZARYA)')[1].split('\r\n')[1:3])
print('Using TLE:')
print(iss_tle)

# Get Wildfire's battery and flight computer blocks for querying later
wildfire_battery = wildfire_agent_template.Battery.get_first()
wildfire_flight_computer = wildfire_agent_template.Component.get(WILDFIRE_FLIGHT_COMPUTER_ID)

# Update Wildfire's orbit
wildfire_agent = scenario.Agent.get(WILDFIRE_AGENT_ID)
wildfire_agent.orbit.update(
    initialStateDefType='TLE',
    initialStateDefParams={ 'tle': iss_tle },
)

# Update simulation clock config
start = now_mjd()
clock = scenario.ClockConfig.get(scenario.data['clockConfig'])
clock.update(startTime=start, stopTime=start+FUTURE_OFFSET)

# Start simulation
sim_client = sedaro_client.get_sim_client(SCENARIO_BRANCH_ID)
simulation_job = sim_client.start()
print(f'Simulation started: {url(scenario)}')

##### 2. Fetch Results of Running Simulation

Using the Sedaro Results API, fetch all available data from the running simulation. Running this periodically will fetch more data as the simulation progresses.

In [ ]:
def fetch_relevant_results():
    def _fetch(tries=0):
        try:
            return SedaroSimulationResult.get_scenario_latest(API_KEY, SCENARIO_BRANCH_ID, host=HOST)
        except KeyboardInterrupt as e:
            raise e
        except Exception as e:
            print(e)
            if tries < 5:
                print(f'No results yet.  Retrying after 10s... ({tries+1} of 5)', end='\r')
                time.sleep(10)
                return _fetch(tries+1)
            raise ValueError(f'Failed to fetch results after {tries} tries')
    results = _fetch()

    wildfire_results = results.agent('Wildfire')
    root_results = wildfire_results.block('root')
    battery_results = wildfire_results.block(wildfire_battery.id)
    return wildfire_results, root_results, battery_results

wildfire_results, root_results, battery_results = fetch_relevant_results()

View the simulated future of the system by plotting the relevant telemetry series from the simulation.

In [ ]:
def plot_relevant_results():
    display.clear_output(wait=True)
    common = { 'linewidth': 1, 'elapsed_time': False, 'height': 2 }
    root_results.position.eci.plot(**common, ylabel='ECI Position', label=['X', 'Y', 'Z'])
    for pack in wildfire_battery.packs:
        wildfire_results.block(pack.id).temperature.degC.plot(**common, show=False, label=pack.name)
    wildfire_results.block(wildfire_flight_computer.id).temperature.degC.plot(
        **common, 
        ylabel='Temperatures [C]', 
        label=wildfire_flight_computer.name,
        color='g'
    )
    battery_results.soc.plot(**common, ylabel='Battery SoC', color='m')
    battery_results.voltage.plot(**common, ylabel='Battery Voltage [V]', color='c')
    battery_results.current.plot(**common, ylabel='Battery Current [A]')

plot_relevant_results()

##### 3 & 4. Gap-Fill with Simulated Telemetry

Fill the gaps in between real telemetry frames with simulated telemetry from the digital twin.  As new telemetry is received, replace the simulation with a new simulation that is informed by the latest real telemetry frame.

In [ ]:
tlm = None
for i in range(120):

    now = now_mjd()

    # Fetch more data if necessary
    current_start = battery_results.soc.mjd[0]
    current_stop = battery_results.soc.mjd[-1]
    if (current_stop-now)/(current_stop-current_start) < 0.1:
        print('Fetching additional data...')
        print('(If you are seeing this frequently, consider increasing `FUTURE_OFFSET`)')
        wildfire_results, root_results, battery_results = fetch_relevant_results()

    sim_tlm = [
        battery_results.soc.value_at(now, interpolate=True), 
        battery_results.voltage.value_at(now, interpolate=True),
        battery_results.current.value_at(now, interpolate=True),
        wildfire_results.block(wildfire_flight_computer.id).temperature.degC.value_at(now, interpolate=True),
        *[wildfire_results.block(pack.id).temperature.degC.value_at(now, interpolate=True) for pack in wildfire_battery.packs]
    ]

    if i == 5:
        display.clear_output(wait=True)
        print('New telemetry received. Updating simulation...')

        # TODO: Inject real telemetry here
        # The following is a placeholder for real telemetry
        tlm = [t*(1-random()/30) for t in sim_tlm]
        tlm_timestamp = now

        # Compare telemetry to simulation
        errors = [abs((s-a)/a) for a, s in zip(tlm, sim_tlm)]

        # If the simulation is still running, first terminate it
        try:
            response = sim_client.terminate(simulation_job['id'])
            print('Previous simulation terminated')
        except:
            print('Previous simulation already terminated')

        # Update Wildfire's initial state to the prior simulation's state at the new telemetry's
        # timestamp merged with all new telemetry.
        wildfire_model = wildfire_results.model_at(tlm_timestamp)
        wildfire_model['blocks'][wildfire_battery.id]['soc'] = tlm[0]
        wildfire_model['blocks'][wildfire_battery.id]['voltage'] = tlm[1]
        wildfire_model['blocks'][wildfire_battery.id]['current'] = tlm[2]
        wildfire_model['blocks'][wildfire_flight_computer.id]['temperature'] = {'degC': tlm[3]}
        wildfire_model['blocks'][wildfire_battery.packs[0].id]['temperature'] = {'degC': tlm[4]}
        wildfire_model['blocks'][wildfire_battery.packs[1].id]['temperature'] = {'degC': tlm[5]}
        wildfire_agent.update(differentiatingState=wildfire_model)

        # Update the simulation clock to start at the new telemetry's 
        # timestamp and start the new simulation.
        print('Starting new simulation with updated initial state...')
        clock.update(startTime=now, stopTime=now + FUTURE_OFFSET)
        simulation_job = sim_client.start()
        print(f'Simulation started {url(scenario)}')

        # Fetch new results
        wildfire_results, root_results, battery_results = fetch_relevant_results()

    display.clear_output(wait=True)
    if tlm:
        summarize_tlm(tlm_timestamp, *tlm, 'Actual')
        print('\nDeviation of simulated telemetry from actual telemetry:')
        print(f' - Max Error: {round(max(errors), 3)*100}%')
        print(f' - Average Error: {round(sum(errors)/len(errors), 3)*100}%')
        print('\n')
    summarize_tlm(now, *sim_tlm, 'Simulated')
    print(f'View Current Simulation: {url(scenario)}')

    time.sleep(0.5)